Tutorial from https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219

There are two main approaches for feature engineering:
* Transformation: take a substring or compute the log a number (feature)
* Aggregation: compute some kind of statistic (min, max, avg, ...)

In [5]:
import pandas as pd
import featuretools as ft

In [2]:
clients = pd.read_csv('./data/clients.csv')
loans = pd.read_csv('./data/loans.csv')
payments = pd.read_csv('./data/payments.csv')

In [7]:
clients.head()

client_id      joined  income  credit_score
0      46109  2002-04-16  172677           527
1      49545  2007-11-14  104564           770
2      41480  2013-03-11  122607           585
3      46180  2001-11-06   43851           562
4      25707  2006-10-06  211422           621

In [8]:
loans.head()

client_id loan_type  loan_amount  repaid  loan_id  loan_start    loan_end  \
0      46109      home        13672       0    10243  2002-04-16  2003-12-20   
1      46109    credit         9794       0    10984  2003-10-21  2005-07-17   
2      46109      home        12734       1    10990  2006-02-01  2007-07-05   
3      46109      cash        12518       1    10596  2010-12-08  2013-05-05   
4      46109    credit        14049       1    11415  2010-07-07  2012-05-21   

   rate  
0  2.15  
1  1.25  
2  0.68  
3  1.24  
4  3.13

In [9]:
payments.head()

loan_id  payment_amount payment_date  missed
0    10243            2369   2002-05-31       1
1    10243            2439   2002-06-18       1
2    10243            2662   2002-06-29       0
3    10243            2268   2002-07-20       0
4    10243            2027   2002-07-31       1

In [4]:
# example of an aggregation using pandas

stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']

stats = clients.merge(stats, left_on='client_id', right_index=True, how='left')

stats.head(10)

client_id      joined  income  credit_score  mean_loan_amount  \
0      46109  2002-04-16  172677           527       8951.600000   
1      49545  2007-11-14  104564           770      10289.300000   
2      41480  2013-03-11  122607           585       7894.850000   
3      46180  2001-11-06   43851           562       7700.850000   
4      25707  2006-10-06  211422           621       7963.950000   
5      39505  2011-10-14  153873           610       7424.050000   
6      32726  2006-05-01  235705           730       6633.263158   
7      35089  2010-03-01  131176           771       6939.200000   
8      35214  2003-08-08   95849           696       7173.555556   
9      48177  2008-06-09  190632           769       7424.368421   

   max_loan_amount  min_loan_amount  
0            14049              559  
1            14971             3851  
2            14399              811  
3            14081             1607  
4            13913             1212  
5            14575              904  
6            14802              851  
7            13194              773  
8            14767              667  
9            14740              659

An Entity is just a table/DataFrame
An EntitySet is a collection of Entities and their relationships

In [10]:
# empty entity set
es = ft.EntitySet(id='clients')

# add the client entity to the entity set
es = es.entity_from_dataframe(entity_id='clients', dataframe=clients,
                              index='client_id', time_index='joined')

es = es.entity_from_dataframe(entity_id='loans', dataframe=loans,
                              index='loan_id', time_index='loan_start')

es = es.entity_from_dataframe(entity_id='payments', dataframe=payments,
                              variable_types={'missed': ft.variable_types.Categorical},
                              make_index=True,
                              index='payments_id',
                              time_index='payment_date')

In [11]:
es['payments']

Entity: payments
  Variables:
    payments_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

In [13]:
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

es = es.add_relationship(r_client_previous)

r_payments = ft.Relationship(es['loans']['loan_id'],
                             es['payments']['loan_id'])

es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [15]:
(features, feature_names) = ft.dfs(entityset=es, target_entity='clients',
                                   agg_primitives=['mean', 'max', 'percent_true', 'last'],
                                   trans_primitives=['years', 'month', 'subtract', 'divide'])

features

income  credit_score  MEAN(loans.loan_amount)  MEAN(loans.repaid)  \
client_id                                                                      
25707      211422           621              7963.950000            0.400000   
26326      227920           633              7270.062500            0.500000   
26695      174532           680              7824.722222            0.388889   
26945      214516           806              7125.933333            0.400000   
29841       38354           523              9813.000000            0.555556   
32726      235705           730              6633.263158            0.578947   
32885       58955           642              9920.400000            0.533333   
32961      230341           714              7882.235294            0.529412   
35089      131176           771              6939.200000            0.600000   
35214       95849           696              7173.555556            0.444444   
38537      127183           643              8986.352941            0.588235   
39384      191204           617              7865.473684            0.631579   
39505      153873           610              7424.050000            0.450000   
41472      152214           638              7510.812500            0.500000   
41480      122607           585              7894.850000            0.500000   
42320      229481           563              7062.066667            0.600000   
44387      151903           781              7387.733333            0.600000   
44601      156341           518              6869.250000            0.750000   
46109      172677           527              8951.600000            0.550000   
46180       43851           562              7700.850000            0.500000   
46958      225709           644              9378.384615            0.230769   
48177      190632           769              7424.368421            0.421053   
49068      128813           603              7617.888889            0.722222   
49545      104564           770             10289.300000            0.700000   
49624       49036           800              9174.533333            0.600000   

           MEAN(loans.rate)  MAX(loans.loan_amount)  MAX(loans.repaid)  \
client_id                                                                
25707              3.477000                   13913                  1   
26326              2.517500                   13464                  1   
26695              2.466111                   14865                  1   
26945              2.855333                   14593                  1   
29841              3.445000                   14837                  1   
32726              3.058947                   14802                  1   
32885              2.436000                   14162                  1   
32961              3.930588                   14784                  1   
35089              3.513500                   13194                  1   
35214              3.108333                   14767                  1   
38537              2.389412                   14804                  1   
39384              3.538421                   14654                  1   
39505              3.190500                   14575                  1   
41472              3.981250                   13657                  1   
41480              3.110500                   14399                  1   
42320              2.457333                   13887                  1   
44387              2.806667                   14900                  1   
44601              4.151000                   14104                  1   
46109              3.152500                   14049                  1   
46180              3.502500                   14081                  1   
46958              3.153846                   14942                  1   
48177              3.938947                   14740                  1   
49068              3.095556                   13910                  1   
